In [1]:
import numpy as np
import pandas as pd
from w2v_utils import *
words, word_to_vec_map = read_glove_vecs('glove.6B.100d.txt')
from collections import Counter
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm_notebook as tqdm
from tqdm import trange


In [15]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\Saloni PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\brown.zip.
[nltk_data]    | Downloading package brown_tei to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roa

[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\pros_cons.zip.
[nltk_data]    | Downloading package qc to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\qc.zip.
[nltk_data]    | Downloading package reuters to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package rte to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\rte.zip.
[nltk_data]    | Downloading package semcor to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package senseval to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\senseval.zip.
[nltk_data]    | Downloading package sentiwordnet to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\sentiwordnet.zip.


[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package perluniprops to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping misc\perluniprops.zip.
[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\Saloni PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\nonbreaking_prefixes.zip.
[nltk_data]    | Downloading package vader_lexicon to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package porter_test to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping stemmers\porter_test.zip.
[nltk_data]    | Downloading package wmt15_eval to C:\Users\Saloni
[nltk_data]    |     PC\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping models\wmt15_eval.zip.
[nltk_data]    | Downloading package mwa_ppdb to C:\Users\Saloni
[n

True

In [13]:
import pickle
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    # Compute the dot product between u and v
    dot = np.dot(u, v)
    # Compute the L2 norm of u
    norm_u = np.sqrt(np.sum(u * u))
    
    # Compute the L2 norm of v
    norm_v = np.sqrt(np.sum(v * v))
    # Compute the cosine similarity
    cosine_similarity = dot / (norm_u * norm_v)
    
    return cosine_similarity



In [3]:
def extractEntityNames(tree, _entityNames=None):
    """
    Creates a local list to hold nodes of tree passed through, extracting named
    entities from the chunked sentences.

    """
    if _entityNames is None:
        _entityNames = []
    try:
        label = tree.label()
    except AttributeError:
        pass
    else:
        if label == 'NE':
            _entityNames.append(' '.join([child[0] for child in tree]))
        else:
            for child in tree:
                extractEntityNames(child, _entityNames=_entityNames)
    return _entityNames




In [4]:
def buildDict(chunkedSentences):
    """
    Uses the global entity list, creating a new dictionary with the properties
    extended by the local list, without overwriting.

    """
    _entityNames = []

    for tree in chunkedSentences:
        extractEntityNames(tree, _entityNames=_entityNames)

    return _entityNames


In [5]:
def chunkSentences(text):
    """
    Parses text into parts of speech tagged with parts of speech labels.
    """
    m_count, f_count = 0,0
    #print(m_count)
    #print(f_count)
    sentences = nltk.sent_tokenize(text)
    tokenizedSentences = [nltk.word_tokenize(sentence)
                          for sentence in sentences]
    for sentence in tokenizedSentences:
        for w in sentence:
            if w.lower() in male_words:
                m_count+=1
            elif w.lower() in female_words:
                f_count+=1
            else:
                pass
    taggedSentences = [nltk.pos_tag(sentence)
                       for sentence in tokenizedSentences]
    chunkedSentences = nltk.ne_chunk_sents(taggedSentences, binary=True)
    return (chunkedSentences,m_count,f_count)




In [6]:
def removeStopwords(entityNames, customStopWords=None):
    """
    Brings in stopwords and custom stopwords to filter mismatches out.
    """
    # Memoize custom stop words
    for name in entityNames:
        if name in stopwords.words('english'):
            entityNames.remove(name)

def getMajorCharacters(entityNames):
    """
    Adds names to the major character list if they appear frequently.
    """
    return {name for name in entityNames if entityNames.count(name) > 5}


In [11]:
!pip install nltk

You should consider upgrading via the 'c:\users\saloni pc\anaconda3\python.exe -m pip install --upgrade pip' command.


In [16]:
def readText():
    """
    Reads the text from a text file.
    """
    with open("Harry Potter and the Sorcerer.txt", "r") as f:
        text = f.read()
    return text
male_words=set(['guy','spokesman','chairman',"men's",'men','him',"he's",'his','boy','boyfriend','boyfriends','boys','brother','brothers','dad','dads','dude','father','fathers','fiance','gentleman','gentlemen','god','grandfather','grandpa','grandson','groom','he','himself','husband','husbands','king','male','man','mr','nephew','nephews','priest','prince','son','sons','uncle','uncles','waiter','widower','widowers'])
female_words=set(['heroine','spokeswoman','chairwoman',"women's",'actress','women',"she's",'her','hers','aunt','aunts','bride','daughter','daughters','female','fiancee','girl','girlfriend','girlfriends','girls','goddess','granddaughter','grandma','grandmother','herself','ladies','lady','lady','mom','moms','mother','mothers','mrs','ms','niece','nieces','priestess','princess','queens','she','sister','sisters','waitress','widow','widows','wife','wives','woman'])
g = word_to_vec_map['woman'] - word_to_vec_map['man']# vector $g$ roughly encodes the concept of "gender".

with open('corpus_dict.pickle', 'rb') as handle:
    hp = pickle.load(handle)
d={}
fileslist=[]
male_count=[]
female_count=[]
for book in tqdm(hp, desc='Progress'):
    for chapter in tqdm(hp[book], postfix=book):
        #print(chapter)
        text = hp[book][chapter]
        chunkedSentences,m_count,f_count = chunkSentences(text)
        entityNames = (buildDict(chunkedSentences))
        # print(entityNames)
        removeStopwords(entityNames)
        majorCharacters = getMajorCharacters(entityNames)
        #print(majorCharacters)
        #print(m_count)
        #print(f_count)
        for w in majorCharacters:
            if w.split()[0].lower() not in words:
                pass
            elif (cosine_similarity(word_to_vec_map[w.split()[0].lower()], g) < 0):
                m_count+=1
            else:
                f_count+=1
        fileslist.append(chapter.split('.')[0])
        male_count.append(m_count)
        female_count.append(f_count)

d['file']=fileslist
d['male_count'] = male_count
d['female_count'] = female_count
df = pd.DataFrame(data=d)
df['male_count'] = (df['male_count']-df['male_count'].mean())/(df['male_count'].max()-df['male_count'].min())
df['female_count'] = (df['female_count']-df['female_count'].mean())/(df['female_count'].max()-df['female_count'].min())
print(df.head())
with open('gender_dict.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)



                       file  male_count  female_count
0                   pg15895   -0.107486     -0.109654
1                   pg33945   -0.001378     -0.023623
2                    pg1392   -0.127530     -0.164698
3  pg19337DickensChristCarl   -0.078988     -0.155239
4                   pg43190    0.110572      0.174208
